In [9]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

from config import env
import data.data_source as data_source

DROP_SQL = "DROP VIEW IF EXISTS ir_cone_diagnostics;"

UPSERT_SQL = """
CREATE OR REPLACE VIEW ir_cone_diagnostics AS
WITH cone_pivot AS (
  SELECT
    curve_date,
    tenor_num,
    model_type,
    days_forward,
    MAX(CASE WHEN cone_type = '1%'  THEN rate END) AS p01,
    MAX(CASE WHEN cone_type = '5%'  THEN rate END) AS p05,
    MAX(CASE WHEN cone_type = '10%' THEN rate END) AS p10,
    MAX(CASE WHEN cone_type = '50%' THEN rate END) AS p50,
    MAX(CASE WHEN cone_type = '90%' THEN rate END) AS p90,
    MAX(CASE WHEN cone_type = '95%' THEN rate END) AS p95,
    MAX(CASE WHEN cone_type = '99%' THEN rate END) AS p99
  FROM rate_cones
  WHERE
    curve_type = 'US Treasury Par'
    AND days_forward IN (30, 90)
  GROUP BY curve_date, tenor_num, model_type, days_forward
),
joined AS (
  SELECT
    c.curve_date + (c.days_forward || ' days')::interval AS target_date,
    c.days_forward,
    c.tenor_num,
    c.model_type,
    c.p01, c.p05, c.p10, c.p50, c.p90, c.p95, c.p99,
    r.curve_date AS realized_date,
    r.rate AS realized_rate
  FROM cone_pivot c
  LEFT JOIN LATERAL (
    SELECT r2.curve_date, r2.tenor_num, r2.rate
    FROM rate_curves r2
    WHERE r2.curve_date >= c.curve_date + (c.days_forward || ' days')::interval
      AND r2.curve_type = 'US Treasury Par'
      AND ROUND(r2.tenor_num::numeric, 6) = ROUND(c.tenor_num::numeric, 6)
    ORDER BY r2.curve_date
    LIMIT 1
  ) r ON TRUE
)
SELECT
  realized_date,
  days_forward,
  tenor_num,
  model_type,
  realized_rate,
  p01, p05, p10, p50, p90, p95, p99,
  CASE
    WHEN realized_rate IS NULL THEN NULL
    WHEN realized_rate < p01 THEN '<1%'
    WHEN realized_rate < p05 THEN '1%-5%'
    WHEN realized_rate < p10 THEN '5%-10%'
    WHEN realized_rate < p50 THEN '10%-50%'
    WHEN realized_rate = p50 THEN '50%'
    WHEN realized_rate <= p90 THEN '50%-90%'
    WHEN realized_rate <= p95 THEN '90%-99%'
    WHEN realized_rate <= p99 THEN '>99%'
    ELSE 'above 99%'
  END AS percentile_band,
  CASE
    WHEN realized_rate IS NULL THEN NULL
    ELSE (realized_rate - p50)
  END AS forecast_error,
  CASE
    WHEN p95 != p05 THEN (realized_rate - p50) / (p95 - p05)
    ELSE NULL
  END AS scaled_error,
  CASE
    WHEN realized_rate < p05 OR realized_rate > p95 THEN 'outside'
    ELSE 'inside'
  END AS cone_hit
FROM joined;
"""

# ─── MAIN ─────────────────────────────────────────────────────────────────────
def main():
    ds = data_source.get_data_source()
    ds.query(DROP_SQL)
    ds.query(UPSERT_SQL)
    print("Setup completed successfully.")

if __name__ == "__main__":
    main()


getting data source for sandbox
Setup completed successfully.
